In [1]:
import os
from dotenv import load_dotenv, dotenv_values

API KEYS

In [3]:
load_dotenv(
    dotenv_path="D:/Data Science Projects/Scalence Internship ML/Bukhari-Muslim-RAG/.env",
    override=True
)

LS_TOKEN = os.environ["LS_TOKEN"]
print(LS_TOKEN)
OpenAI_TOKEN = os.environ["OpenAI_TOKEN"]
print(OpenAI_TOKEN)

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
sk-proj-mO7ul2Am4ejmD4YNZMW7T3BlbkFJdG1j5bof6oHms0qYMlOZ


In [4]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = LS_TOKEN

DOCUMENTS LOAD

In [5]:
from langchain.document_loaders import TextLoader

loader = TextLoader("story_sherlock_holmes.txt")

docs = loader.load()

In [6]:
# docs

SPLIT WITH TEXT SPLITTER

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [8]:
# splits

MAKE A VECTOR STORE AND STORE THE STORY WITH EMBEDDINGS

In [9]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [10]:
collection_name = 'sherlock_holmes_collection'
persist_directory = "D:/Data Science Projects/Scalence Internship ML/Bukhari-Muslim-RAG/chroma"

In [12]:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding = OpenAIEmbeddings(api_key=OpenAI_TOKEN),
    collection_name = collection_name,
    persist_directory = persist_directory
)

In [13]:
retriever = vectorstore.as_retriever()

MAKE PROMPT FOR CHAIN

In [14]:
#### RETRIEVAL and GENERATION ####
from langchain import hub

# Prompt
prompt = hub.pull("rlm/rag-prompt")


In [15]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

MAKE LLM FOR CHAIN

In [17]:
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    api_key=OpenAI_TOKEN
)


In [20]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

MAKE A RAG CHAIN

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

MAKE A QUERY USING THE CHAIN

In [22]:
# Question
rag_chain.invoke("Who is Neville St. Clair?")

'Neville St. Clair is a character who is feared to have entered a vile murder-trap on the riverside and may not leave it alive.'